In [15]:
import uvicorn
from fastapi import FastAPI, File, UploadFile, HTTPException  
import requests
import os
from cv2 import imread, resize
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import torch
import torchvision.models as models
from torchvision import transforms

In [16]:
loaded_model = torch.load(f="./model/resnet50_1.pt")
model = models.resnet50(num_classes=2,pretrained=False)
model.load_state_dict(loaded_model)

c:\Users\vosco\anaconda3\envs\cloud\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vosco\anaconda3\envs\cloud\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [17]:
def predict(img_url: str):
    # try:
        open('img.jpeg', 'wb').write(requests.get(img_url).content)
        img = imread('img.jpeg')
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((256, 256), antialias=True)
        ])

        img_tensor = transform(img).unsqueeze(0)
        model.eval()
        # models.resnet50 = model
        with torch.no_grad():
            output = model(img_tensor)

        _, predicted_idx = torch.max(output, 1)

        dict_value = {'0': 'leo', '1': 'orion'}
        class_predict = dict_value[str(int(predicted_idx > 0.5))]

        probability = torch.softmax(output, dim=1)[0]
        predicted_probability = probability[predicted_idx].item()

        result = {
            "Prediction": str(class_predict),
            "Probabilité": str(predicted_probability)
        }

        return result

In [18]:
predict("https://github.com/heidisbk/MuspellheimGnistor/blob/main/constellations/leo/leo0.png?raw=true")

{'Prediction': 'leo', 'Probabilité': '0.5916783213615417'}